In [26]:
import pandas as pd
import os
import json
import numpy as np
import re

In [27]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\Private Dining and Meeting Rooms\PrivateDiningAndMeetingRooms.json")
anytimeLounge_config = json.load(config_file)

In [28]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [29]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData):
    
    allHoursOfOperation = []
    
    for i in range(0, len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}

        season = "All Season"
        ## adding season value
        if season and not pd.isnull(season):
            seasonValue = {}
            seasonValue["value"] = season
            hoursOfOperationValue["Season"] = [seasonValue]
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw):
                startTime = str(startTimeRaw).strip()
            
            if pd.notnull(endTimeRaw):
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else :  
                addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue
            
        hoursOfOperationForEachRow.append(hoursOfOperation)
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [30]:
## Getting the output File name
output_fileName = anytimeLounge_config['outputFileName']

common_column_indexes = []
column_names = []

## getting common column information and anytimeLounge course related column information from config file
common_column_count = len(anytimeLounge_config['commonColumnInformation'])

## Getting column index from config json file
for i in range(0, common_column_count):
    common_column_indexes.append(anytimeLounge_config['commonColumnInformation'][i]['columnIndex'])
    column_names.append(anytimeLounge_config['commonColumnInformation'][i]['outputColumnName'])
    
## getting Hours of Related column information from config file
hoursOfOperationInfo = anytimeLounge_config['HoursOfOperationInformation']


## extract Data for any time lounge
anytimeLounge = surveyData.iloc[:,common_column_indexes]
anytimeLounge.columns = column_names

## Adding Hours of Operation
hrsOfOperation = prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData)
anytimeLounge['HrsOfOpeartion'] = hrsOfOperation

## Adding 3 information together
anytimeLounge['Name']='Private Dining and Meeting Rooms'
anytimeLounge['Index']=1
anytimeLounge['Status'] = 'Active'
anytimeLounge['Type'] = 'Private Events & Meeting Rooms'
anytimeLounge['AVEquipment'] = True

## condition for filtering out those clubs that does not have this amenity
filter_condition = anytimeLounge["Private Dining & Meeting Rooms (Total for all Rooms)"] > 0

## deleting all the rows for which name is blank or, Na or, 0
dataExceptBlankName = anytimeLounge[filter_condition]
dataExceptBlankName.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/Private Dining and Meeting Rooms/" + output_fileName)

## getting those clubs that does not have this amenity
clubsThatDoNotHaveAnyTimeLounge = anytimeLounge[~filter_condition]
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('Status', 1)
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('Type', 1)
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('HrsOfOpeartion', 1)
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('Index', 1)
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('Name', 1)
clubsThatDoNotHaveAnyTimeLounge = clubsThatDoNotHaveAnyTimeLounge.drop('AVEquipment', 1)

clubsThatDoNotHaveAnyTimeLounge.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/Private Dining and Meeting Rooms/PrivateDiningAndMeetingRooms_ClubsNotHaving.csv")

C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\akk\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o